# 01 — Data Exploration (EDA)

In this notebook we explore the **HAM10000** dataset to understand:
- How many images per class (class distribution)
- What the images look like (sample grid)
- Image dimensions and properties
- Class imbalance — and why it matters for medical data

> **Dataset:** HAM10000 — 10,015 dermatoscopic images of 7 skin lesion types

---

## How to Run This Notebook

**Option A — Kaggle (Recommended, no download needed):**
1. Go to Kaggle → New Notebook
2. Click "Add Data" → search "Skin Cancer MNIST: HAM10000" → Add
3. Copy-paste the cells from this notebook
4. The dataset will be at `/kaggle/input/skin-cancer-mnist-ham10000/`

**Option B — Locally:**
1. Download HAM10000 from Kaggle
2. Extract into `data/HAM10000/`
3. Run this notebook from the `notebooks/` folder

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
from collections import Counter

# ── Detect environment ──────────────────────────────────
# Kaggle sets this env variable automatically
IS_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

if IS_KAGGLE:
    DATA_DIR = Path('/kaggle/input/skin-cancer-mnist-ham10000')
    RESULTS_DIR = Path('/kaggle/working/results')
    print('Running on KAGGLE')
else:
    sys.path.append('..')
    DATA_DIR = Path('../data/HAM10000')
    RESULTS_DIR = Path('../results')
    print('Running LOCALLY')

RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# ── Verify data dir exists ──────────────────────────────
if DATA_DIR.exists():
    print(f'Data dir: {DATA_DIR}')
    print(f'Contents: {os.listdir(DATA_DIR)[:10]}')
else:
    print(f'WARNING: Data dir not found at {DATA_DIR}')
    print('Make sure you added the "Skin Cancer MNIST: HAM10000" dataset!')

# ── Class definitions ───────────────────────────────────
CLASS_NAMES = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
CLASS_LABELS = {
    'akiec': 'Actinic Keratosis',
    'bcc': 'Basal Cell Carcinoma',
    'bkl': 'Benign Keratosis',
    'df': 'Dermatofibroma',
    'mel': 'Melanoma',
    'nv': 'Melanocytic Nevus',
    'vasc': 'Vascular Lesion',
}

sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 120

print('Setup complete!')

---
## 1. Load Metadata

In [ ]:
# Load the metadata CSV
# On Kaggle the file is directly in the input folder
metadata_path = DATA_DIR / 'HAM10000_metadata.csv'

# Fallback: search for it if not at expected path
if not metadata_path.exists():
    csv_files = list(DATA_DIR.glob('*.csv'))
    print(f'Available CSVs: {csv_files}')
    metadata_path = csv_files[0] if csv_files else metadata_path

df = pd.read_csv(metadata_path)

print(f'Total samples: {len(df)}')
print(f'Columns: {list(df.columns)}')
print()
df.head(10)

In [ ]:
# Basic info
print('Data types:')
print(df.dtypes)
print()
print('Missing values:')
print(df.isnull().sum())
print()
print('Unique values per column:')
for col in df.columns:
    print(f'  {col}: {df[col].nunique()}')

---
## 2. Class Distribution

This is **the most important chart** for understanding our data.
Medical datasets are almost always imbalanced — some conditions are much more common than others.

In [ ]:
# Count per class
class_counts = df['dx'].value_counts()
print('Samples per class:')
for cls, count in class_counts.items():
    label = CLASS_LABELS.get(cls, cls)
    pct = count / len(df) * 100
    print(f'  {label:30s}  ({cls})  →  {count:5d}  ({pct:.1f}%)')

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
labels = [CLASS_LABELS[c] for c in class_counts.index]
colors = sns.color_palette('husl', len(class_counts))

bars = axes[0].barh(labels, class_counts.values, color=colors)
axes[0].set_xlabel('Number of Images')
axes[0].set_title('Class Distribution — HAM10000')
axes[0].invert_yaxis()

# Add count labels on bars
for bar, count in zip(bars, class_counts.values):
    axes[0].text(bar.get_width() + 50, bar.get_y() + bar.get_height()/2,
                 f'{count}', va='center', fontsize=10)

# Pie chart
axes[1].pie(class_counts.values, labels=labels, autopct='%1.1f%%',
            colors=colors, startangle=90)
axes[1].set_title('Class Proportions')

plt.tight_layout()
fig.savefig(RESULTS_DIR / 'class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\nSaved to {RESULTS_DIR / "class_distribution.png"}')

### Key Observation: Class Imbalance

Notice how **Melanocytic Nevus (nv)** dominates the dataset (~67%), while **Dermatofibroma** and **Vascular Lesion** have very few samples.

This is a classic medical data problem. We'll handle it in the training notebook using:
- **Weighted loss function** (penalize misclassifying rare classes more)
- **Data augmentation** (generate more samples for underrepresented classes)
- **Stratified splitting** (ensure all classes are represented in train/val/test)

---
## 3. Sample Images

Let's see what each class actually looks like.

In [ ]:
# Find where the images are
# Kaggle: images might be in subfolders or flat in the input dir
# Local: HAM10000 splits images across two folders

possible_dirs = [
    DATA_DIR / 'HAM10000_images_part_1',
    DATA_DIR / 'HAM10000_images_part_2',
    DATA_DIR,  # Kaggle sometimes puts everything flat
]

# Keep only dirs that actually exist
image_dirs = [d for d in possible_dirs if d.exists()]
print(f'Image directories found: {image_dirs}')

def find_image(image_id):
    """Find an image across all image directories."""
    for d in image_dirs:
        path = d / f'{image_id}.jpg'
        if path.exists():
            return path
    return None

# Verify images exist
sample = df['image_id'].iloc[0]
path = find_image(sample)
if path:
    print(f'\nFound sample image: {path}')
    img = Image.open(path)
    print(f'Image size: {img.size}')
    print(f'Image mode: {img.mode}')
else:
    # Last resort: find any jpg in the data dir
    all_jpgs = list(DATA_DIR.rglob('*.jpg'))
    print(f'\nDirect lookup failed. Found {len(all_jpgs)} jpgs total.')
    if all_jpgs:
        # Figure out which subdirectory they're in
        parent = all_jpgs[0].parent
        print(f'Images are in: {parent}')
        image_dirs = [parent]
        path = find_image(sample)
        if path:
            img = Image.open(path)
            print(f'Image size: {img.size}')
            print(f'Image mode: {img.mode}')
    else:
        print('No images found — check your data folder!')

In [ ]:
# Show sample images per class (3 samples each)
n_samples = 3
fig, axes = plt.subplots(len(CLASS_NAMES), n_samples, figsize=(12, 4 * len(CLASS_NAMES)))

for row, cls in enumerate(CLASS_NAMES):
    class_df = df[df['dx'] == cls].sample(n=n_samples, random_state=42)

    for col, (_, sample) in enumerate(class_df.iterrows()):
        img_path = find_image(sample['image_id'])
        if img_path:
            img = Image.open(img_path)
            axes[row, col].imshow(img)

        if col == 0:
            axes[row, col].set_ylabel(CLASS_LABELS[cls], fontsize=11, fontweight='bold')

        axes[row, col].set_xticks([])
        axes[row, col].set_yticks([])

plt.suptitle('Sample Images per Class', fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
fig.savefig(RESULTS_DIR / 'sample_images.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Image Properties

In [ ]:
# Check image dimensions for a random sample
sample_df = df.sample(n=200, random_state=42)
widths = []
heights = []

for _, row in sample_df.iterrows():
    img_path = find_image(row['image_id'])
    if img_path:
        img = Image.open(img_path)
        widths.append(img.size[0])
        heights.append(img.size[1])

print(f'Image dimensions (sample of {len(widths)}):')
print(f'  Width  — min: {min(widths)}, max: {max(widths)}, mean: {np.mean(widths):.0f}')
print(f'  Height — min: {min(heights)}, max: {max(heights)}, mean: {np.mean(heights):.0f}')
print(f'  Most common size: {Counter(zip(widths, heights)).most_common(1)[0]}')

---
## 5. Patient Analysis

Important: some patients have **multiple images**. We need to make sure images from the same patient don't end up in both train and test sets (data leakage!).

In [ ]:
if 'lesion_id' in df.columns:
    unique_lesions = df['lesion_id'].nunique()
    print(f'Total images: {len(df)}')
    print(f'Unique lesions: {unique_lesions}')
    print(f'Avg images per lesion: {len(df) / unique_lesions:.1f}')
    print()

    # Distribution of images per lesion
    imgs_per_lesion = df['lesion_id'].value_counts()
    print('Images per lesion:')
    print(imgs_per_lesion.describe())
else:
    print('No lesion_id column — skipping patient analysis')

---
## 6. Age & Sex Distribution (if available)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Age distribution
if 'age' in df.columns:
    df['age'].dropna().hist(bins=30, ax=axes[0], color='steelblue', edgecolor='white')
    axes[0].set_xlabel('Age')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Age Distribution')
    axes[0].axvline(df['age'].median(), color='red', linestyle='--', label=f'Median: {df["age"].median():.0f}')
    axes[0].legend()

# Sex distribution
if 'sex' in df.columns:
    sex_counts = df['sex'].value_counts()
    axes[1].bar(sex_counts.index, sex_counts.values, color=['#4C72B0', '#DD8452'])
    axes[1].set_ylabel('Count')
    axes[1].set_title('Sex Distribution')

plt.tight_layout()
plt.show()

---
## 7. Localization Distribution

In [ ]:
if 'localization' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))

    loc_counts = df['localization'].value_counts()
    loc_counts.plot(kind='barh', ax=ax, color='steelblue')
    ax.set_xlabel('Number of Images')
    ax.set_title('Lesion Localization (Body Part)')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print('No localization column')

---
## Summary

Key findings from our EDA:

1. **Class imbalance** — Melanocytic Nevus dominates, rare classes need special handling
2. **Image dimensions** — All images are the same size, we'll resize to 224x224 for our model
3. **Duplicate lesions** — Same lesion can appear multiple times, need careful train/test splitting
4. **Demographics** — Dataset includes age, sex, and body location metadata

### Next Steps

→ Move on to **02_model_training.ipynb** to build and train our classification model!